# Advent of Code 2024 Day 14 

### Setup

In [1]:
from aocd import get_data, submit

day = 14
year = 2024


In [ ]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

In [ ]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

##### Data Parsing

In [ ]:
import numpy as np

def parse_data(raw_data:str):
    lines = raw_data.split('\n')

    robots = []
    for line in lines:
        pos_data, velo_data, *_ = line.split(' ')
        pos = [(int(y), int(x)) for x, y in [pos_data[2:].strip().split(',')]][0]
        velo = [(int(y), int(x)) for x, y in [velo_data[2:].strip().split(',')]][0]

        robots.append({'position': np.array(pos), 'velocity': np.array(velo)})

    return robots
        

sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data

### Part One!

In [5]:
use_sample_data = False
part = 'a'

In [ ]:
data = sample_data if use_sample_data else test_data

data

In [7]:
def generate_puzzle_map(row_size, col_size, default_value=0):
    return np.ones((row_size, col_size), dtype=int) * default_value

In [8]:
def get_next_robot_position(current_position, velocity, max_row, max_col):
    next_pos = current_position + velocity

    # wrap around
    if next_pos[0] < 0:
        next_pos[0] = max_row - abs(next_pos[0])
    
    elif next_pos[0] >= max_row:
        next_pos[0] = next_pos[0] - max_row
    
    if next_pos[1] < 0:
        next_pos[1] = max_col - abs(next_pos[1])

    elif next_pos[1] >= max_col:
        next_pos[1] = next_pos[1] - max_col
    
    return next_pos

In [9]:
def simulate(puzzle, robots, seconds=100):
    for _ in range(seconds):
        for robot in robots:
            robot['position'] = get_next_robot_position(robot['position'], robot['velocity'], puzzle.shape[0], puzzle.shape[1])
        
    return robots

In [10]:
def plot_robots(puzzle, robots):
    puzzle = puzzle.copy()
    for robot in robots:
        puzzle[robot['position'][0], robot['position'][1]] += 1
    
    return puzzle

In [11]:
def get_puzzle_quadrants(puzzle:np.ndarray):
    puzzle = puzzle.copy()
    row_size, col_size = puzzle.shape

    # drop middle vertical and horizontal if not even 
    if row_size % 2 != 0:
        puzzle = np.delete(puzzle, row_size // 2, axis=0)
        row_size -= 1
    
    if col_size % 2 != 0:
        puzzle = np.delete(puzzle, col_size // 2, axis=1)
        col_size -= 1
    
    # split into quadrants
    top_left = puzzle[:row_size//2, :col_size//2]
    top_right = puzzle[:row_size//2, col_size//2:]
    bottom_left = puzzle[row_size//2:, :col_size//2]
    bottom_right = puzzle[row_size//2:, col_size//2:]

    return {
        'top_left': top_left,
        'top_right': top_right,
        'bottom_left': bottom_left,
        'bottom_right': bottom_right
    }


In [12]:
def calculate_safety_score(puzzle:np.ndarray):
    quadrants = get_puzzle_quadrants(puzzle)

    quad_sums = []
    for _, quad in quadrants.items():
        quad_sums.append(quad.sum())
    
    # multiply all quad_sums together 
    return np.prod(quad_sums)

In [ ]:
data_copy = [ {'position': x['position'], 'velocity': x['velocity']} for x in data ]
puzzle_shape = (103, 101) if not use_sample_data else (7, 11)
puzzle_map = generate_puzzle_map(*puzzle_shape)

robots = simulate(puzzle_map, data_copy, seconds=100)
plotted_puzzle = plot_robots(puzzle_map, robots)

part_a_answer = calculate_safety_score(plotted_puzzle)
part_a_answer, puzzle_shape

In [ ]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

### Part Two!

In [15]:
use_sample_data = False
part='b'

In [16]:
data = sample_data if use_sample_data else test_data

In [ ]:
data_copy = [ {'position': x['position'], 'velocity': x['velocity']} for x in data ]
puzzle_shape = (103, 101)
puzzle_map = generate_puzzle_map(*puzzle_shape)

with open('part_2.txt', 'w') as fp:    
    np.set_printoptions(threshold=np.inf, linewidth=np.inf)
    for i in range(100_000):
        robots = simulate(puzzle_map, data_copy, seconds=1)
        plotted_puzzle = plot_robots(puzzle_map, robots)

        if np.max(plotted_puzzle) == 1:
            part_b_answer = i + 1 # add 1 to account for 0 indexing
            fp.write(f'{part_b_answer}\n\n')
            fp.write(f'{plotted_puzzle}\n\n')
            break

part_b_answer

In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)